# Projeto de sistema de transmissão para eixos deslocados em 90°

Disciplina: TMEC038 Elementos de Máquinas II

Alunos:
- Marcos Augusto Belizario
- Murasson Obil

## 1. Importando pacotes necessários

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb

from abc import ABC, abstractmethod
from scipy.optimize import differential_evolution, minimize, NonlinearConstraint
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

## 2. Definindo os requisitos do sistema

### 2.1. Definindo as caracteristicas do sistema de transmissão

Elementos de transmissão:
- transmissão da potência para eixos deslocados em 90°; <br>
- seleção de um par de engrenagens adequado; <br>
- as engrenagens cônicas foram selecionados em favor das engrenagens helicoidais devido às características de rotação e potência do sistema, com uma probabilidade de custo menor. <br>

Potências transmitidas:
- para o projeto de resistência dos materiais, é considerado o caso de maiores tensões; <br>
- maiores tensões quando a potência é transmitida integralmente (eficiência de 100%); <br>
- esse caso ideal não está longe da realidade devido à alta eficiência de transmissão de torque por engrenagens; <br>
- resultado dad potências de entrada e de saída de 5.0 kW. <br>

Número de ciclos:
- 3 anos com dois turnos de 8 horas diárias; <br>
- 52 semanas em um ano, porém 4 semanas de férias; <br>
- 6 dias de trabalho semanais; <br>
- resultado de 829,440 horas; <br>
- resultado de 1,492,992,000 ciclos com rotação de 1800 rpm no pinhão. <br>

Confiabilidade das engrenagens:
- o sistema deve ter confiabilidade combinada de ao menos 95%; <br>
- as engrenagens, junto com os eixos, são os componentes mais caros do sistema; <br>
- priorização da confiabilidade das engrenagens; <br>
- define-se a confiabilidade das engrenagens de 99.9%. <br>

Coefiente de segurança do conjunto:
- coeficiente de segurança de 1.75; <br>
- coeficiente de segurança elevado pois o custo do conjunto de engrenagens é caro. <br>

In [2]:
# Transmission characteristics
sys_rpm_out = 660.0 / 2.0
sys_power_out = 2.5e3
sys_ratio = 2.0
sys_life_cycle = 1.0e7
sys_reliability = 0.95
sys_cs = 1.5

### 2.2. Definindo as caracteristicas das engrenagens

A partir do cátalogo da fabricante de engrenagens KHK, definimos

Ângulo de pressão $\alpha$:
- ângulo de pressão padrâo de 20°. <br>

Fator de sobrecarga $K_0$:
- acionamento uniforme com choques moderados: 1.25. <br>

Número de qualidade de AGMA $Q_v$:
- qualidade alta fornecida pela KHK: 3. <br>

Fator de distribuição de carga $K_{mb}$:
- apenas uma engrenagem entre mancais devido à limitações de espaço; <br>
- projeto conservador: 1.40. <br>

Fator de curvatura ao longo do comprimento $K_x$:
- engrenagens cônicas retas: 1.0. <br>

Fator de coroamento $K_{xc}$:
- engrenagens coroadas: 1.5. <br>

Material:
- aço carbono AISI 1045 endurecido por completo grau 1 com dureza de 194 HB; <br>
- escolha do material com maior versatilidade (maior quantidade de módulo distintos). <br>

<img src="./tab-13-2.png" alt="Tabela 13.2" style="height: 400px;"/>
<img src="./fig-13-7.png" alt="Figura 13.7" style="height: 400px;"/>

In [3]:
# Gears characteristics
gear_alpha = 20.0
gear_k0 = 1.25
gear_qv = 7.0
gear_kmb = 1.3
gear_kx = 1.0
gear_kxc = 2.0

# Material characteristics
# Aco endurecido por completo grau 2 (220 HB)
mat_hardness = 220.0
mat_sigma_sf = (0.30 * mat_hardness + 14.48) * 1.0e6
mat_sigma_sp = (2.35 * mat_hardness + 162.89) * 1.0e6
mat_youngs = 200.0e9
mat_poisson = 0.30

## 3. Cálculo dos parâmetros geométricos e de resistência dos materiais

Classe abstrata $Gear$ que define o comportamento de uma engrenagem qualquer.

Classe $ConicalGear$ que define o comportamento de uma engrenagem cônica a partir da classe $Gear$.

In [13]:
class Gear(ABC):
    """Abstract base class representing a generic gear."""
    def __init__(self, module, nteeth, width, alpha, qv):
        """
        Initialize the Gear with basic parameters.

        Parameters
        ----------
        module : float
            Module of the gear.
        nteeth : int
            Number of teeth.
        width : float
            Face width of the gear.
        alpha : float
            Pressure angle in degrees.
        qv : float
            AGMA quality number.
        """
        self._width = None

        self.module = module
        self.nteeth = nteeth
        self.width = width
        self.alpha = alpha
        self.qv = qv

    @property
    @abstractmethod
    def width(self):
        """
        Get the face width of the gear.

        Returns
        -------
        float
            Face width of the gear.
        """
        return self._width

    @width.setter
    @abstractmethod
    def width(self, value):
        """
        Set the face width of the gear.

        Parameters
        ----------
        value : float
            New face width.
        """
        raise NotImplementedError()

    @property
    @abstractmethod
    def diam_eq(self):
        """
        Get the equivalent diameter of the gear.

        Returns
        -------
        float
            Equivalent diameter.
        """
        raise NotImplementedError()

    @abstractmethod
    def ks(self):
        """
        Calculate the size factor.

        Returns
        -------
        float
            Size factor.
        """
        raise NotImplementedError()

    @abstractmethod
    def km(self):
        """
        Calculate the mesh factor.

        Returns
        -------
        float
            Mesh factor.
        """
        raise NotImplementedError()

    @abstractmethod
    def stress_agma(self, k0, j, torque, rpm):
        """
        Calculate the AGMA stress by the bending criteria.

        Parameters
        ----------
        k0 : float
            Overload factor.
        j : float
            Geometry factor.
        torque : float
            Torque applied.
        rpm : float
            Revolutions per minute.

        Returns
        -------
        float
            AGMA stress by the bending criteria.
        """
        raise NotImplementedError()

    @abstractmethod
    def stress_agma_pressure(self, k0, kxc, i, youngs, poisson, ratio, torque, rpm):
        """
        Calculate AGMA stress by the pressure criteria.

        Parameters
        ----------
        k0 : float
            Overload factor.
        kxc : float
            Crowning factor.
        i : float
            Geometry factor.
        youngs : float
            Young's module.
        poisson : float
            Poisson coefficient.
        ratio : float
            Gear ratio.
        torque : float
            Torque applied.
        rpm : float
            Revolutions per minute.

        Returns
        -------
        float
            AGMA stress by the pressure criteria.
        """
        raise NotImplementedError()

    @staticmethod
    @abstractmethod
    def yn(n):
        """
        Calculate the stress cycle factor.

        Parameters
        ----------
        n : float
            Number of cycles.

        Returns
        -------
        float
            Stress cycle factor.
        """
        raise NotImplementedError()

    @abstractmethod
    def csf(self, st, n, temp, r, k0, j, torque, rpm):
        """
        Calculate the safety factor by the strength criteria.

        Parameters
        ----------
        st : float
            AGMA strength.
        n : float
            Number of cycles.
        temp : float
            Temperature.
        r : float
            Reliability.
        k0 : float
            Overload factor.
        j : float
            Geometry factor.
        torque : float
            Torque applied.
        rpm : float
            Revolutions per minute.

        Returns
        -------
        float
            Safety factor by the strength criteria.
        """
        raise NotImplementedError()

    @abstractmethod
    def csp(self, k0, kxc, i, sc, youngs, poisson, ratio, torque, rpm, n, temp, r):
        """
        Calculate the safety factor by the pressure criteria.

        Parameters
        ----------
        k0 : float
            Overload factor.
        kxc : float
            Crowning factor.
        i : float
            Geometry factor.
        sc : float
            AGMA strength for the pressure criteria.
        youngs : float
            Young's module.
        poisson : float
            Poisson coefficient.
        ratio : float
            Gear ratio.
        torque : float
            Torque applied.
        rpm : float
            Revolutions per minute.
        n : float
            Number of cycles.
        temp : float
            Temperature.
        r : float
            Reliability.

        Returns
        -------
        float
            Safety factor by the pressure criteria.
        """
        raise NotImplementedError()

    @property
    def diam_pitch(self):
        """
        Get the pitch diameter.

        Returns
        -------
        float
            Pitch diameter.
        """
        return self.module * self.nteeth

    def tangencial_force(self, torque):
        """
        Calculate the tangential force for a given torque.

        Parameters
        ----------
        torque : float
            Applied torque.

        Returns
        -------
        float
            Tangential force.
        """
        return 2.0 * torque / self.diam_eq

    def tangencial_velocity(self, rpm):
        """
        Calculate the tangential velocity for a given rpm.

        Parameters
        ----------
        rpm : float
            Revolutions per minute.

        Returns
        -------
        float
            Tangential velocity.
        """
        return self.diam_eq * rpm * np.pi / 60.0

    def kv(self, rpm):
        """
        Calculate the velocity factor.

        Parameters
        ----------
        rpm : float
            Revolutions per minute.

        Returns
        -------
        float
            Velocity factor.
        """
        aux_b = 0.25 * (12.0 - gear_qv) ** (2.0 / 3.0)
        aux_a = 50.0 + 56.0 * (1.0 - aux_b)
        return (1.0 + np.sqrt(200.0 * self.tangencial_velocity(rpm)) / aux_a) ** aux_b

    @staticmethod
    def yt(temp):
        """
        Calculate temperature factor.

        Parameters
        ----------
        temp : float
            Temperature in degrees Celsius.

        Returns
        -------
        float
            Temperature factor.
        """
        if temp < 120.0:
            return 1.0
        else:
            return (273.0 + temp) / 393.0

    @staticmethod
    def yz(r):
        """
        Calculate reliability factor.

        Parameters
        ----------
        r : float
            Reliability.

        Returns
        -------
        float
            Reliability factor.
        """
        if r < 0.99:
            return 0.658 - 0.0759 * np.log(1.0 - r)
        elif r < 0.9999:
            return 0.5 - 0.109 * np.log(1.0 - r)

        raise ValueError("r must be between 0.5 and 0.9999")


class ConicalGear(Gear):
    """Represents a conical gear, inheriting from the Gear class."""
    def __init__(self, module, nteeth, width, alpha, qv, kx, gamma):
        """
        Initialize the ConicalGear with specific parameters.

        Parameters
        ----------
        module : float
            Module of the gear.
        nteeth : int
            Number of teeth.
        width : float
            Face width of the gear.
        alpha : float
            Pressure angle in degrees.
        qv : float
            AGMA quality number.
        kx : float
            Axial load factor specific to conical gear.
        gamma : float
            Cone angle in degrees.
        """
        self.gamma = gamma
        self.kx = kx
        super().__init__(module, nteeth, width, alpha, qv)

    @property
    def width(self):
        """
        Get the face width of the gear

        Returns
        -------
        float
            Face width.
        """
        return self._b

    @width.setter
    def width(self, value):
        """
        Set the face width of the gear.

        Parameters
        ----------
        value : float
            New face width.
        """
        if value > 0.5 * self.diam_pitch / np.cos(self.gamma):
            self._b = 0.5 * self.diam_pitch / np.cos(self.gamma)
        else:
            self._b = value

    @property
    def diam_mean(self):
        """
        Calculate the mean diameter of the conical gear.

        Returns
        -------
        float
            Mean diameter.
        """
        return self.diam_pitch - self.width * np.sin(self.gamma)

    @property
    def diam_eq(self):
        """
        Get the mean diameter as the pitch diameter for conical gears.

        Returns
        -------
        float
            Mean diameter.
        """
        return self.diam_mean

    def ks(self):
        """
        Calculate the size factor for conical gears.

        Returns
        -------
        float
            Size factor.
        """
        if self.module < 1.6e-3:
            return 0.5
        elif self.module < 50.0e-3:
            return 0.4867 + 0.008339 * self.module

    def km(self):
        """
        Calculate the gear mesh factor for conical gears.

        Returns
        -------
        float
            Gear mesh factor.
        """
        return gear_kmb + 5.6 * self.width ** 2.0

    def stress_agma(self, k0, j, torque, rpm):
        """
        Calculate AGMA stress by the bending criteria for conical gears.

        Parameters
        ----------
        k0 : float
            Overload factor.
        j : float
            Geometry factor.
        torque : float
            Torque applied.
        rpm : float
            Revolutions per minute.

        Returns
        -------
        float
            AGMA stress by the bending criteria.
        """
        force_factors = k0 * self.kv(rpm) * self.ks() * self.km()
        geometry_factors = self.kx * j
        return force_factors * self.tangencial_force(torque) / (geometry_factors * self.width * self.module)

    def stress_agma_pressure(self, k0, kxc, i, youngs, poisson, ratio, torque, rpm):
        """
        Calculate AGMA stress by the pressure criteria for conical gears.

        Parameters
        ----------
        k0 : float
            Overload factor.
        kxc : float
            Crowning factor.
        i : float
            Geometry factor.
        youngs : float
            Young's module.
        poisson : float
            Poisson coefficient.
        ratio : float
            Gear ratio.
        torque : float
            Torque applied.
        rpm : float
            Revolutions per minute.

        Returns
        -------
        float
            AGMA stress by the pressure criteria.
        """
        self.width = 60.0e-3
        cp = np.sqrt(1.0 / (2.0 * np.pi * (1.0 - poisson ** 2.0) / youngs))

        cs = 0.0
        if self.width < 12.7e-3:
            cs = 0.5
        elif self.width < 114.3e-3:
            cs = 0.4375 + 0.00492 * self.width * 1.0e3
        else:
            cs = 1.0

        print(f"{k0 = } | {self.tangencial_force(torque) = } | {self.kv(rpm) = } | {cs = } | {kxc = } | {self.km() = } | {self.width = } | {self.diam_pitch = } | {i = }")
        return cp * np.sqrt(k0 * self.tangencial_force(torque) * self.kv(rpm) * cs * kxc * self.km() / (self.width * self.diam_pitch * i))

    @staticmethod
    def yn(n):
        """
        Calculate stress cycle factor for life cycles.

        Parameters
        ----------
        n : float
            Number of cycles.

        Returns
        -------
        float
            Stress cycle factor.
        """
        if n < 3.0e6:
            return 6.1514 * n ** -0.1192
        else:
            return 1.683 * n ** -0.0323

    def csf(self, st, n, temp, r, k0, j, torque, rpm):
        """
        Calculate the safety factor by the strength criteria.

        Parameters
        ----------
        st : float
            AGMA strength.
        n : float
            Number of cycles.
        temp : float
            Temperature.
        r : float
            Reliability.
        k0 : float
            Overload factor.
        j : float
            Geometry factor.
        torque : float
            Torque applied.
        rpm : float
            Revolutions per minute.

        Returns
        -------
        float
            Safety factor by the strength criteria.
        """
        return st * self.yn(n) / (Gear.yt(temp) * Gear.yz(r) * self.stress_agma(k0, j, torque, rpm))

    def csp(self, k0, kxc, i, sc, youngs, poisson, ratio, torque, rpm, n, temp, r):
        """
        Calculate the safety factor by the pressure criteria for conical gears.

        Parameters
        ----------
        k0 : float
            Overload factor.
        kxc : float
            Crowning factor.
        i : float
            Geometry factor.
        sc : float
            AGMA strength for the pressure criteria.
        youngs : float
            Young's module.
        poisson : float
            Poisson coefficient.
        ratio : float
            Gear ratio.
        torque : float
            Torque applied.
        rpm : float
            Revolutions per minute.
        n : float
            Number of cycles.
        temp : float
            Temperature.
        r : float
            Reliability.

        Returns
        -------
        float
            Safety factor by the pressure criteria.
        """
        ch = 1.0
        zn = 3.4822 * n ** -0.0602
        return sc * zn * ch / (Gear.yt(temp) * Gear.yz(r) * self.stress_agma_pressure(k0, kxc, i, youngs, poisson, ratio, torque, rpm))

In [14]:
def calc_states(power=None, torque=None, rpm=None):
    """
    Calculate the missing state (power, torque, or rpm) based on the provided values.

    Parameters
    ----------
    power : float, optional
        Power in watts.
    torque : float, optional
        Torque in Newton-meter (Nm).
    rpm : float, optional
        Revolutions per minute (RPM).

    Returns
    -------
    tuple(float)
        A tuple containing calculated values for (power, torque, rpm).
    """
    if power is None:
        power = torque * (rpm * np.pi / 30.0)
    elif torque is None:
        torque = power / (rpm * np.pi / 30)
    elif rpm is None:
        rpm = (30.0 / np.pi) * power / torque

    return power, torque, rpm


def form_factor(nteeth, alpha=20.0):
    """
    Calculate the form factor using the number of teeth and pressure angle.

    Parameters
    ----------
    nteeth : int
        Number of teeth.
    alpha : float, optional
        Pressure angle in degrees (default is 20.0).

    Returns
    -------
    float
        Form factor based on the number of teeth and pressure angle.
    """
    z_data = np.array([10, 11, 12, 13, 14,
                       15, 16, 17, 18, 19,
                       20, 22, 24, 26, 28,
                       30, 32, 34, 36, 38,
                       40, 45, 50, 55, 60,
                       65, 70, 75, 80, 90,
                       100, 150, 200, 300],
                      dtype=np.int32)

    # Form factor for pressure angle of 14.5 degrees
    y_145 = np.array([0.176, 0.192, 0.210, 0.223, 0.236,
                      0.245, 0.255, 0.264, 0.270, 0.277,
                      0.283, 0.292, 0.302, 0.308, 0.314,
                      0.318, 0.322, 0.325, 0.329, 0.332,
                      0.336, 0.340, 0.346, 0.352, 0.355,
                      0.358, 0.360, 0.361, 0.363, 0.366,
                      0.368, 0.375, 0.378, 0.382],
                     dtype=np.float64)

    # Form factor for pressure angle of 20.0 degrees
    y_200 = np.array([0.201, 0.226, 0.245, 0.264, 0.276,
                      0.289, 0.295, 0.302, 0.308, 0.314,
                      0.320, 0.330, 0.337, 0.344, 0.352,
                      0.358, 0.364, 0.370, 0.377, 0.383,
                      0.389, 0.399, 0.408, 0.415, 0.421,
                      0.425, 0.429, 0.433, 0.436, 0.442,
                      0.446, 0.458, 0.463, 0.471],
                     dtype=np.float64)

    # Interpolate the number of teeth
    y_145_pred = np.interp(nteeth, z_data, y_145)
    y_200_pred = np.interp(nteeth, z_data, y_200)

    # Interpolate the pressure angle
    return np.interp(alpha, [14.5, 20.0], [y_145_pred, y_200_pred])


def lewis_equation(module, nteeth, alpha, k0, ft, sigma_esc, cs=1.0):
    """
    Calculate the face width necessary to support the force 'ft' with safety coefficient 'cs' of a gear using the Lewis equation.

    Parameters
    ----------
    module : float
        Module of the gear.
    nteeth : int
        Number of teeth.
    alpha : float
        Pressure angle in degrees.
    k0 : float
        Overload factor.
    ft : float
        Tangential force.
    sigma_esc : float
        Allowable stress.
    cs : float, optional
        Safety coefficient. Default: 1.0.

    Returns
    -------
    float
        Face width necessary to support the conditions.
    """
    return cs * (k0 * ft) / (module * form_factor(nteeth, alpha) * sigma_esc)  # return b


def generate_gears_lewis(module, nteeth_p, alpha, qv, k0, kx, sigma_esc, cs, ratio, torque_p, b0=1.0e-3, rtol=1.0e-6, niter=100):
    """
    Generate pinion and crown gears based on the Lewis equation and input parameters.

    Parameters
    ----------
    module : float
        Initial module of the gears.
    nteeth_p : int
        Initial number of teeth of the pinion.
    alpha : float
        Pressure angle in degrees.
    qv : float
        Quality number.
    k0 : float
        Overload factor.
    kx : float
        Curvature factor.
    sigma_esc : float
        Allowable stress.
    cs : float
        Safety coefficient.
    ratio : float
        Gear ratio.
    torque_p : float
        Torque applied to the pinion.
    b0 : float, optional
        Initial face width estimation. Default: 1.0e-3.
    rtol : float, optional
        Tolerance for iteration termination. Default: 1.0e-6.
    niter : int, optional
        Number of iterations. Default: 100.

    Returns
    -------
    tuple
        Tuple containing the pinion and crown ConicalGear objects.
    """
    pinion = ConicalGear(module, nteeth_p, b0, alpha, qv, kx, np.arctan(1.0 / ratio))
    crown = ConicalGear(module, round(ratio * pinion.nteeth), b0, alpha, qv, kx, np.arctan(ratio))

    b1 = b0
    for _ in range(niter):
        b2 = min(lewis_equation(module, nteeth_p, alpha, k0, pinion.tangencial_force(torque_p), sigma_esc, cs),
                 0.5 * pinion.diam_pitch / np.cos(pinion.gamma))
        pinion.width = b2
        crown.width = b2

        if b2 - b1 < b1 * rtol:
            break

        b1 = b2

    return pinion, crown

# 4. Função custo de aquisição do par de engrenagens

Com o objetivo de reduzir o custo da transmissão, define-se uma função do custo do par de engrenagens em função das suas características.

### 4.1.  Definição da função custo

Função de regressão do custo obtida a partir dos dados de engrenagens da fabricante KHK utilizando o método XGBoost.

Dados obtidos de [] e disponíveis em: .

In [6]:
df_gear = pd.read_csv('data/bevel-gear-khk.csv')
df_material = pd.read_csv('data/material-hardness.csv')

df_gear = df_gear.merge(df_material[['Material', 'Hardness']], on='Material', how='left')
df_gear = df_gear[df_gear['Material'] != 'Nylon']

text_cols = ['Material']
for col in [col for col in df_gear.columns if col not in text_cols]:
    df_gear[col] = df_gear[col].astype(str).str.replace(r"[^0-9.]", "", regex=True).astype(np.float64)

df_gear['VolumeUpperBound'] = np.pi * (df_gear['PitchDiameter'] / 2.0) ** 2.0 * df_gear['FaceWidth']

df_gear.head()

,GearRatio,MetricModule,NumberOfTeeth,BoreDiameter,FaceWidth,PitchDiameter,HubDiameter,HubProjection,Material,Price,PricePaired,Hardness,VolumeUpperBound
0,1.5,1.5,20.0,8.0,9.0,30.0,25.0,8.8,Black Oxide Coated Carbon Steel,39.29,54.12,194.0,6361.725124
1,1.5,1.5,30.0,10.0,9.0,45.0,30.0,8.0,Black Oxide Coated Carbon Steel,54.12,39.29,194.0,14313.881528
2,1.5,2.0,20.0,10.0,11.0,40.0,30.0,11.7,Black Oxide Coated Carbon Steel,51.22,70.53,194.0,13823.007676
3,1.5,2.0,30.0,10.0,11.0,60.0,35.0,15.0,Black Oxide Coated Carbon Steel,70.53,51.22,194.0,31101.767271
4,1.5,2.5,20.0,12.0,15.0,50.0,35.0,12.5,Black Oxide Coated Carbon Steel,63.18,94.95,194.0,29452.431127


In [7]:
# Define feature and target columns
features = ['GearRatio', 'MetricModule', 'NumberOfTeeth', 'PitchDiameter', 'FaceWidth', 'Hardness', 'VolumeUpperBound']
target = 'PricePaired'

X = df_gear[features]
y = df_gear[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model using XGBoost
model = xgb.XGBRegressor(
    n_estimators=1000,
    eta=0.05,
    max_depth=4,
    subsample=0.5,
    grow_policy='lossguide',
    eval_metric='rmse',
)

model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error Scaled: {mse / y_test.mean():.4f}")
print(f"R-squared: {r2:.4f}")

Mean Squared Error Scaled: 18.8613
R-squared: 0.8780


### 4.2. Funcões custo para geração das engrenagens

cost_width: Calcula a função de custo da largura para otimizar a largura da face das engrenagens para satisfazer o coeficiente de segurança necessário com base nas características do pinhão e da coroa.

generate_and_optimize_gears: Gera e otimiza o pinhão e coroa com a largura necessária para satisfazer o coeficiente de segurança.

cost_function: Calcula o custo do par de engrenagens com base nas características das engrenagens.

cost_function_opt: Gera e otimiza o pinhão e coroa com a largura necessária para satisfazer o coeficiente de segurança e calcula o custo do par com base nas características das engrenagens.

cost_function_opt_int: Gera e otimiza o pinhão e coroa com a largura necessária para satisfazer o coeficiente de segurança e calcula o custo do par com base nas características das engrenagens, com o módulo restrito a valores comerciais

In [8]:
def cost_width(width, pinion, crown, k0, kxc, j_p, j_c, i, sigma_esc_sf, sigma_esc_sp, youngs, poisson, cs, ratio, torque_p, rpm_p, life_cycle, reliability):
    """
    Calculate the width cost function to optimize the face width of the gears to satisfy the required safety coefficient based on pinion and crown characteristics.
    The pressure criteria has a 0.5 increase in its safety coefficient compared to the bending criteria.

    Parameters
    ----------
    width : array-like
        Initial width for optimization.
    pinion : Gear
        Pinion gear object.
    crown : Gear
        Crown gear object.
    k0 : float
        Overload factor.
    kxc : float
        Crowning factor.
    j_p : float
        Geometry factor for the pinion by the bending criteria.
    j_c : float
        Geometry factor for the crown by the bending criteria.
    i : float
        Geometry factor by the pressure criteria.
    sigma_esc_sf : float
        Allowable stress by the bending criteria.
    sigma_esc_sp : float
        Allowable stress by the pressure criteria.
    youngs : float
        Young's module.
    poisson : float
        Poisson coefficient.
    cs : float
        Safety coefficient.
    ratio : float
        Gear ratio.
    torque_p : float
        Applied torque to the pinion.
    rpm_p : float
        RPM of the pinion.
    life_cycle : float
        Total life cycles expected for the system.
    reliability : float
        Desired reliability.

    Returns
    -------
    float
        The cost function value for optimization.
    """
    pinion.width = width[0]
    crown.width = width[0]

    pinion_csf = pinion.csf(sigma_esc_sf, life_cycle, 25.0, reliability, k0, j_p, torque_p, rpm_p)
    crown_csf = crown.csf(sigma_esc_sf, life_cycle / ratio, 25.0, reliability, k0, j_c, torque_p * ratio, rpm_p / ratio)

    pinion_csp = pinion.csp(k0, kxc, i, sigma_esc_sp, youngs, poisson, ratio, torque_p, rpm_p, life_cycle, 25.0, reliability)
    crown_csp = pinion.csp(k0, kxc, i, sigma_esc_sp, youngs, poisson, ratio, torque_p * ratio / ratio, rpm_p, life_cycle, 25.0, reliability)

    return (min(pinion_csf, crown_csf, pinion_csp + 0.5, crown_csp + 0.5) - cs) ** 2.0


def generate_and_optimize_gears(module, nteeth_p, alpha, qv, k0, kx, kxc, j_p, j_c, i, sigma_esc_sf, sigma_esc_sp, youngs, poisson, cs, ratio, torque_p, rpm_p, life_cycle, reliability, rtol=1.0e-6, niter=100):
    """
    Generate and optimize pinion and crown gear widths given the input parameters.

    Parameters
    ----------
    module : float
        Metric module of the gears.
    nteeth_p : int
        Number of teeth on the pinion.
    alpha : float
        Pressure angle of the gears in degrees.
    qv : float
        Quality number of the gears.
    k0 : float
        Overload factor.
    kx : float
        Alignment factor.
    kxc : float
        Crowning factor.
    j_p : float
        Geometry factors by the bending criteria for the pinion.
    j_c : float
        Geometry factors by the bending criteria for the crown.
    i : float
        Geometry factor by the pressure criteria.
    sigma_esc_sf : float
        Allowable stress by the bending criteria.
    sigma_esc_sp : float
        Allowable stress by the pressure criteria.
    youngs : float
        Young's module.
    poisson : float
        Poisson coefficient.
    cs : float
        Safety coefficient.
    ratio : float
        Gear ratio.
    torque_p : float
        Torque applied to the pinion.
    rpm_p : float
        RPM applied to the pinion.
    life_cycle : float
        Total expected life cycles of the system.
    reliability : float
        Desired reliability factor.
    rtol : float, optional
        Relative tolerance for the iteration process (default: 1.0e-6).
    niter : int, optional
        Maximum number of iterations (default: 100).

    Returns
    -------
    tuple(Gear)
        Optimized pinion and crown gear objects.
    """
    pinion, crown = generate_gears_lewis(module, nteeth_p, alpha, qv, k0, kx, sigma_esc_sf, cs, ratio, torque_p, rtol, niter)

    result = minimize(
        cost_width,
        x0=np.array([pinion.width]),
        args=(pinion, crown, k0, kxc, j_p, j_c, i, sigma_esc_sf, sigma_esc_sp, youngs, poisson, cs, ratio, torque_p, rpm_p, life_cycle, reliability),
        bounds=[(1.0e-3, 0.5 * pinion.diam_pitch / np.cos(pinion.gamma))],
        method='SLSQP',
        options={'maxiter': 1000, 'ftol': 1e-9}
    )

    pinion.width = result.x[0]
    crown.width = result.x[0]

    return pinion, crown


def cost_function(pinion, ratio, hardness):
    """
    Calculate the gear pair cost based on the gear characteristics.

    Parameters
    ----------
    pinion : Gear
        Pinion gear.
    ratio : float
        Gear ratio.
    hardness : float
        Material hardness.

    Returns
    -------
    numpy.ndarray
        Predicted cost of the gear pair.
    """
    metric_module = pinion.module * 1.0e3
    face_width = pinion.width * 1.0e3
    number_of_teeth = pinion.nteeth

    pitch_diameter = metric_module * number_of_teeth
    volume_upper_bound = np.pi * (pitch_diameter / 2.0) ** 2.0 * face_width

    return model.predict([[
        ratio,
        metric_module,
        number_of_teeth,
        pitch_diameter,
        face_width,
        hardness,
        volume_upper_bound
    ]])[0]


def cost_function_opt(x, alpha, qv, k0, kx, kxc, j_p, j_c, i, sigma_esc_sf, sigma_esc_sp, youngs, poisson, cs, ratio, torque_p, rpm_p, life_cycle, reliability, hardness, rtol=1.0e-6, niter=100):
    """
    Calculate the gear pair cost based on the gear characteristics.

    Parameters
    ----------
    x : numpy.ndarray
        Array containing the metric module and number of teeth.
    alpha : float
        Pressure angle of the gears in degrees.
    qv : float
        Quality number of the gears.
    k0 : float
        Overload factor.
    kx : float
        Alignment factor.
    kxc : float
        Crowning factor.
    j_p : float
        Geometry factors by the bending criteria for the pinion.
    j_c : float
        Geometry factors by the bending criteria for the crown.
    i : float
        Geometry factor by the pressure criteria.
    sigma_esc_sf : float
        Allowable stress by the bending criteria.
    sigma_esc_sp : float
        Allowable stress by the pressure criteria.
    youngs : float
        Young's module.
    poisson : float
        Poisson coefficient.
    cs : float
        Safety coefficient.
    ratio : float
        Gear ratio.
    torque_p : float
        Applied torque to the pinion.
    rpm_p : float
        RPM of the pinion.
    life_cycle : float
        Total life cycles expected for the system.
    reliability : float
        Desired reliability.
    hardness : float
        Material hardness.
    rtol : float, optional
        Relative tolerance for the iteration process (default: 1.0e-6).
    niter : int, optional
        Maximum number of iterations (default: 100).

    Returns
    -------
    numpy.ndarray
        Predicted cost of the gear pair.
    """
    module, number_of_teeth = x

    pinion, crown = generate_and_optimize_gears(
        module,
        number_of_teeth,
        alpha,
        qv,
        k0,
        kx,
        kxc,
        j_p,
        j_c,
        i,
        sigma_esc_sf,
        sigma_esc_sp,
        youngs,
        poisson,
        cs,
        ratio,
        torque_p,
        rpm_p,
        life_cycle,
        reliability,
        rtol,
        niter
    )

    return cost_function(pinion, ratio, hardness)


def cost_function_opt_int(x, module_allowed, alpha, qv, k0, kx, kxc, j_p, j_c, i, sigma_esc_sf, sigma_esc_sp, youngs, poisson, cs, ratio, torque_p, rpm_p, life_cycle, reliability, hardness, rtol=1.0e-6, niter=100):
    """
    Calculate the gear pair cost based on the gear characteristics.

    Parameters
    ----------
    x : numpy.ndarray
        Array containing the metric module and number of teeth index.
    module_allowed : dict(float, float)
        List of allowed metric modules in crescent order.
    alpha : float
        Pressure angle of the gears in degrees.
    qv : float
        Quality number of the gears.
    k0 : float
        Overload factor.
    kx : float
        Alignment factor.
    kxc : float
        Crowning factor.
    j_p : float
        Geometry factors by the bending criteria for the pinion.
    j_c : float
        Geometry factors by the bending criteria for the crown.
    i : float
        Geometry factor by the pressure criteria.
    sigma_esc_sf : float
        Allowable stress by the bending criteria.
    sigma_esc_sp : float
        Allowable stress by the pressure criteria.
    youngs : float
        Young's module.
    poisson : float
        Poisson coefficient.
    cs : float
        Safety coefficient.
    ratio : float
        Gear ratio.
    torque_p : float
        Applied torque to the pinion.
    rpm_p : float
        RPM of the pinion.
    life_cycle : float
        Total life cycles expected for the system.
    reliability : float
        Desired reliability.
    hardness : float
        Material hardness.
    rtol : float, optional
        Relative tolerance for the iteration process (default: 1.0e-6).
    niter : int, optional
        Maximum number of iterations (default: 100).

    Returns
    -------
    numpy.ndarray
        Predicted cost of the gear pair.
    """
    module_int, number_of_teeth = x
    module = module_allowed[int(module_int)]

    return cost_function_opt(np.array([module, x[1]]), alpha, qv, k0, kx, kxc, j_p, j_c, i, sigma_esc_sf, sigma_esc_sp, youngs, poisson, cs, ratio, torque_p, rpm_p, life_cycle, reliability, hardness, rtol, niter)

# 5. Aplicando o equacionamento ao estudo de caso de maneira manual

### 5.1. Obtendo os parâmetros iniciais a partir da Equação de Lewis

Para evitar interferência, temos que $z_p >= 15$, e para obter a quantidade de dentes na coroa seja inteira com uma relação de transmissão de 1.5, temos que $z_p$ deve ser divísivel por 2.
Portanto, o menor número de dentes para o pinhão é $z_1 = 16$.

Determinamos um módulo inicial de 6.0 mm forma arbitrária e obtemos a espessura necessária pela Equação de Lewis.

<img src="./fig-13-6.png" alt="Figura 13.6" style="height: 400px;"/>|
<img src="./fig-13-9.png" alt="Figura 13.9" style="height: 400px;"/>

In [15]:
m0 = 6.0e-3
z10 = 20

power1, torque1, rpm1 = calc_states(sys_power_out, None, sys_rpm_out * sys_ratio)
power2, torque2, rpm2 = calc_states(sys_power_out, None, sys_rpm_out)

pinion, crown = generate_gears_lewis(m0, z10, gear_alpha, gear_qv, gear_k0, gear_kx, mat_sigma_sf, sys_cs, sys_ratio, torque1)

j1 = 0.213
pinion_sigma_agma = pinion.stress_agma(gear_k0, j1, torque1, rpm1)
pinion_csf = pinion.csf(mat_sigma_sf, sys_life_cycle, 25.0, sys_reliability, gear_k0, j1, torque1, rpm1)

j2 = 0.179
crown_sigma_agma = crown.stress_agma(gear_k0, j2, torque2, rpm2)
crown_csf = crown.csf(mat_sigma_sf, sys_life_cycle / sys_ratio, 25.0, sys_reliability, gear_k0, j2, torque2, rpm2)

gear_i = 0.078

print(f"Pinhao: "
      f"m = {pinion.module * 1.0e3:.2f} mm | "
      f"z = {pinion.nteeth} | "
      f"b = {pinion.width * 1.0e3:.2f} mm | "
      f"dp = {pinion.diam_pitch * 1.0e3:.2f} mm | "
      f"dm = {pinion.diam_mean * 1.0e3:.2f} mm | "
      f"sigma_agma = {pinion_sigma_agma * 1.0e-6:.3f} MPa | "
      f"CSf = {pinion_csf:.3f}")
print(f" Coroa: "
      f"m = {crown.module * 1.0e3:.2f} mm | "
      f"z = {crown.nteeth} | "
      f"b = {crown.width * 1.0e3:.2f} mm | "
      f"dp = {crown.diam_pitch * 1.0e3:.2f} mm | "
      f"dm = {crown.diam_mean * 1.0e3:.2f} mm | "
      f"sigma_agma = {crown_sigma_agma * 1.0e-6:.3f} MPa | "
      f"CSf = {crown_csf:.3f}")

Pinhao: m = 6.00 mm | z = 20 | b = 7.53 mm | dp = 120.00 mm | dm = 116.63 mm | sigma_agma = 66.479 MPa | CSf = 1.367
 Coroa: m = 6.00 mm | z = 40 | b = 7.53 mm | dp = 240.00 mm | dm = 233.27 mm | sigma_agma = 79.107 MPa | CSf = 1.175


### 5.2. Obtenção da espessura em contato do par de engrenagens

A equação de Lewis fornece uma espessura inicial para o projeto se dado um módulo conhecido, com a espessura adequada devendo ser obtida a partir da equação de AGMA de forma a respeitar o coeficiente de segurança definido.

A equação de AGMA pode ser utilizada de forma análitica, porém aqui podemos utilizar a implementação já feita para os cálculos de tensão de flexão e coeficiente de segurança, encontrando a espessura mínima que resulte no coeficiente de segurança adequado a partir de métodos de otimização. A otimização é feita de forma que

$$
\begin{equation}
\begin{aligned}
\min_{b} \quad & [min(CS_{f,pinion}, CS_{f,crown}) - CS_{required}]²\\
\end{aligned}
\end{equation}
$$

Para um coeficiente de segurança de 1.75, temos

$$
\begin{equation}
\begin{aligned}
\min_{b} \quad & [min(CS_{f,pinion}, CS_{f,crown}) - 1.75]²\\
\end{aligned}
\end{equation}
$$

In [16]:
result = minimize(
    cost_width,
    x0=[pinion.width],
    args=(pinion, crown, gear_k0, gear_kxc, j1, j2, gear_i, mat_sigma_sf, mat_sigma_sp, mat_youngs, mat_poisson, sys_cs, sys_ratio, torque1, rpm1, sys_life_cycle, sys_reliability),
    bounds=[(1.0e-3, 0.5 * pinion.diam_pitch / np.cos(pinion.gamma))],
    method='SLSQP',
    options={'maxiter': 1000, 'ftol': 1e-9}
)

# Set the optimized face width
pinion.width = result.x[0]
crown.width = result.x[0]

pinion_sigma_agma = pinion.stress_agma(gear_k0, j1, torque1, rpm1)
pinion_csf = pinion.csf(mat_sigma_sf, sys_life_cycle, 25.0, sys_reliability, gear_k0, j1, torque1, rpm1)
pinion_csp = pinion.csp(gear_k0, gear_kxc, gear_i, mat_sigma_sp, mat_youngs, mat_poisson, sys_ratio, torque1, rpm1, sys_life_cycle, 25.0, sys_reliability)

crown_sigma_agma = crown.stress_agma(gear_k0, j2, torque2, rpm2)
crown_csf = crown.csf(mat_sigma_sf, sys_life_cycle / sys_ratio, 25.0, sys_reliability, gear_k0, j2, torque2, rpm2)
crown_csp = crown.csp(gear_k0, gear_kxc, gear_i, mat_sigma_sp, mat_youngs, mat_poisson, sys_ratio, torque2, rpm2, sys_life_cycle, 25.0, sys_reliability)

print(f"Espessura otimizada: b = {result.x[0] * 1.0e3:.2f} mm")
print(f"Pinhao: "
      f"m = {pinion.module * 1.0e3:.2f} mm | "
      f"z = {pinion.nteeth} | "
      f"b = {pinion.width * 1.0e3:.2f} mm | "
      f"dp = {pinion.diam_pitch * 1.0e3:.2f} mm | "
      f"dm = {pinion.diam_mean * 1.0e3:.2f} mm | "
      f"CSf = {pinion_csf:.3f} | "
      f"CSp = {pinion_csp:.3f}")
print(f" Coroa: "
      f"m = {crown.module * 1.0e3:.2f} mm | "
      f"z = {crown.nteeth} | "
      f"b = {crown.width * 1.0e3:.2f} mm | "
      f"dp = {crown.diam_pitch * 1.0e3:.2f} mm | "
      f"dm = {crown.diam_mean * 1.0e3:.2f} mm | "
      f"CSf = {crown_csf:.3f} | "
      f"CSp = {crown_csp:.3f}")
print(f"Custo do par: {cost_function(pinion, sys_ratio, mat_hardness):.2f}")

k0 = 1.25 | self.tangencial_force(torque) = 776.4875209838685 | self.kv(rpm) = 1.2721764023332214 | cs = 0.7326999999999999 | kxc = 2.0 | self.km() = 1.32016 | self.width = 0.06 | self.diam_pitch = 0.12 | i = 0.078
k0 = 1.25 | self.tangencial_force(torque) = 776.4875209838685 | self.kv(rpm) = 1.2721764023332214 | cs = 0.7326999999999999 | kxc = 2.0 | self.km() = 1.32016 | self.width = 0.06 | self.diam_pitch = 0.12 | i = 0.078
k0 = 1.25 | self.tangencial_force(torque) = 776.4875209838685 | self.kv(rpm) = 1.2721764023332214 | cs = 0.7326999999999999 | kxc = 2.0 | self.km() = 1.32016 | self.width = 0.06 | self.diam_pitch = 0.12 | i = 0.078
k0 = 1.25 | self.tangencial_force(torque) = 776.4875209838685 | self.kv(rpm) = 1.2721764023332214 | cs = 0.7326999999999999 | kxc = 2.0 | self.km() = 1.32016 | self.width = 0.06 | self.diam_pitch = 0.12 | i = 0.078
k0 = 1.25 | self.tangencial_force(torque) = 776.4875209838685 | self.kv(rpm) = 1.2721764023332214 | cs = 0.7326999999999999 | kxc = 2.0 | se

# 6. Aplicando o equacionamento ao estudo de caso utilizando algoritmos de otimização

In [11]:
def constraint_ratio_exact(x, ratio):
    """
    Calculate the constraint for the number of gear teeth based on the given ratio.

    Parameters
    ----------
    x : array_like
        Array containing the metric module index and the number of teeth.
    ratio : float
        Desired gear transmission ratio.

    Returns
    -------
    float
        Residual of the constraint.
    """
    metric_module_int, number_of_teeth = x
    decimal = number_of_teeth * ratio - math.floor(number_of_teeth * ratio)

    # residual should be maximum at decimal = 0.5 and zero at decimal = 0.0 and decimal = 1.0
    return - (decimal - 0.0) * (decimal - 1.0)


def constraint_cs_int(x, module_allowed, alpha, qv, k0, kx, kxc, j_p, j_c, i, sigma_esc_sf, sigma_esc_sp, youngs, cs, ratio, torque_p, rpm_p, life_cycle, reliability, rtol=1.0e-6, niter=100):
    """
    Calculate the security coefficient constraint for optimization.
    Ensures the security coefficient is greater than a specified threshold.
    The pressure criteria has a 0.5 increase in its safety coefficient compared to the bending criteria.

    Parameters
    ----------
    x : numpy.ndarray
        Array containing the metric module and number of teeth.
    module_allowed : dict(float, float)
        List of allowed metric modules in crescent order.
    alpha : float
        Pressure angle of the gears in degrees.
    qv : float
        Quality number of the gears.
    k0 : float
        Overload factor.
    kx : float
        Alignment factor.
    kxc : float
        Crowning factor.
    j_p : float
        Geometry factors by the bending criteria for the pinion.
    j_c : float
        Geometry factors by the bending criteria for the crown.
    i : float
        Geometry factor by the pressure criteria.
    sigma_esc_sf : float
        Allowable stress by the bending criteria.
    sigma_esc_sp : float
        Allowable stress by the pressure criteria.
    youngs : float
        Young's module.
    cs : float
        Safety coefficient.
    ratio : float
        Gear ratio.
    torque_p : float
        Applied torque to the pinion.
    rpm_p : float
        RPM of the pinion.
    life_cycle : float
        Total life cycles expected for the system.
    reliability : float
        Desired reliability.
    rtol : float, optional
        Relative tolerance for the iteration process (default: 1.0e-6).
    niter : int, optional
        Maximum number of iterations (default: 100).

    Returns
    -------
    float
        Minimum calculated safety coefficient of pinion and crown.
        Ensures it meets or exceeds the required safety coefficient.

    Notes
    -----
    This function prints the specifications of the optimized pinion.
    """

    metric_module_int, number_of_teeth = x
    metric_module = module_allowed[int(metric_module_int)]

    pinion, crown = generate_and_optimize_gears(
        metric_module,
        number_of_teeth,
        alpha,
        qv,
        k0,
        kx,
        kxc,
        j_p,
        j_c,
        i,
        sigma_esc_sf,
        sigma_esc_sp,
        youngs,
        cs,
        ratio,
        torque_p,
        rpm_p,
        life_cycle,
        reliability,
        rtol,
        niter
    )

    pinion_csf = pinion.csf(sigma_esc_sf, life_cycle, 25.0, reliability, k0, j_p, torque_p, rpm_p)
    crown_csf = crown.csf(sigma_esc_sf, life_cycle / ratio, 25.0, reliability, k0, j_c, torque_p * ratio, rpm_p / ratio)

    pinion_csp = pinion.csp(k0, kxc, i, sigma_esc_sp, youngs, ratio, torque_p, rpm_p, rpm_p, life_cycle, 25.0, reliability)
    crown_csp = pinion.csp(k0, kxc, i, sigma_esc_sp, youngs, ratio, torque_p * ratio, rpm_p / ratio, rpm_p, life_cycle, 25.0, reliability)

    return min(pinion_csf, crown_csf, pinion_csp, crown_csp + 0.5, crown_csp + 0.5)

In [12]:
# Define the allowed/commercial values of metric module
module_allowed = [
    1.0e-3,
    1.25e-3,
    1.5e-3,
    2.0e-3,
    2.5e-3,
    3.0e-3,
    4.0e-3,
    5.0e-3,
    6.0e-3,
    8.0e-3,
]


def apply_constraint_ratio_exact(x):
    return constraint_ratio_exact(x, sys_ratio)


def apply_constraint_cs_int(x):
    return constraint_cs_int(
        x,
        module_allowed,
        gear_alpha,
        gear_qv,
        gear_k0,
        gear_kx,
        gear_kxc,
        j1,
        j2,
        gear_i,
        mat_sigma_sf,
        mat_sigma_sp,
        mat_youngs,
        sys_cs,
        sys_ratio,
        torque1,
        rpm1,
        sys_life_cycle,
        sys_reliability,
        rtol=1.0e-6,
        niter=100
    )


# Define bounds for parameters
bounds = [(0, len(module_allowed) - 1), (15, 60)]

# Minimize the cost with the constraint
result = differential_evolution(
    cost_function_opt_int,
    bounds,
    constraints=[
        NonlinearConstraint(apply_constraint_ratio_exact, -1.0e-7, 1.0e-7),
        NonlinearConstraint(apply_constraint_cs_int, sys_cs, np.inf),
    ],
    args=(module_allowed, gear_alpha, gear_qv, gear_k0, gear_kx, j1, j2, mat_sigma_sf, sys_cs, sys_ratio, torque1, rpm1, sys_life_cycle, sys_reliability, mat_hardness),
    integrality=[True, True],
    popsize=50,
    init='halton',
    strategy='best1bin',
    maxiter=1000,
    polish=False
)

print("Parâmetros otimizados:", result.x)
print("Custo mínimo:", result.fun)

k0 = 1.25 | self.tangencial_force(torque) = 8898.364141884722 | self.kv(rpm) = 20.326958108643357 | cs = 0.4377952 | kxc = 2.0 | self.km() = 1.32016 | self.width = 0.06 | self.diam_pitch = 0.15 | i = 0.078
k0 = 1.25 | self.tangencial_force(torque) = 8898.364141884722 | self.kv(rpm) = 20.326958108643357 | cs = 0.4377952 | kxc = 2.0 | self.km() = 1.32016 | self.width = 0.06 | self.diam_pitch = 0.15 | i = 0.078
k0 = 1.25 | self.tangencial_force(torque) = 8898.364141884722 | self.kv(rpm) = 20.326958108643357 | cs = 0.4377952 | kxc = 2.0 | self.km() = 1.32016 | self.width = 0.06 | self.diam_pitch = 0.15 | i = 0.078
k0 = 1.25 | self.tangencial_force(torque) = 8898.364141884722 | self.kv(rpm) = 20.326958108643357 | cs = 0.4377952 | kxc = 2.0 | self.km() = 1.32016 | self.width = 0.06 | self.diam_pitch = 0.15 | i = 0.078
k0 = 1.25 | self.tangencial_force(torque) = 8898.364141884722 | self.kv(rpm) = 1.3398409623518344 | cs = 0.4377952 | kxc = 2.0 | self.km() = 1.32016 | self.width = 0.06 | self.

/tmp/ipykernel_658681/3240738981.py:494: RuntimeWarning: invalid value encountered in sqrt
  cp = np.sqrt(1.0 / (2.0 * np.pi * (1.0 - poisson ** 2.0) / youngs))
/home/belisarius/miniconda3/envs/python310/lib/python3.10/site-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/home/belisarius/miniconda3/envs/python310/lib/python3.10/site-packages/scipy/optimize/_slsqp_py.py:441: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)


RuntimeError: The map-like callable must be of the form f(func, iterable), returning a sequence of numbers the same length as 'iterable'

In [ ]:
pinion, crown = generate_and_optimize_gears(
    module_allowed[int(result.x[0])],
    int(result.x[1]),
    gear_alpha,
    gear_qv,
    gear_k0,
    gear_kx,
    j1,
    j2,
    mat_sigma_sf,
    sys_cs,
    sys_ratio,
    torque1,
    rpm1,
    sys_life_cycle,
    sys_reliability,
)

print(f"Pinhao: "
      f"m = {pinion.module * 1.0e3:.2f} mm | "
      f"z = {pinion.nteeth} | "
      f"b = {pinion.width * 1.0e3:.2f} mm | "
      f"dp = {pinion.diam_pitch * 1.0e3:.2f} mm | "
      f"dm = {pinion.diam_mean * 1.0e3:.2f} mm | "
      f"CSf = {pinion.csf(mat_sigma_sf, sys_life_cycle, 25.0, sys_reliability, gear_k0, j1, torque1, rpm1):.3f}"
      f"CSp = {pinion.csp(gear_k0, gear_kxc, gear_i, mat_sigma_sp, mat_youngs, sys_ratio, torque1, rpm1, rpm1, sys_life_cycle, 25.0, sys_reliability):.3f}")
print(f" Coroa: "
      f"m = {crown.module * 1.0e3:.2f} mm | "
      f"z = {crown.nteeth} | "
      f"b = {crown.width * 1.0e3:.2f} mm | "
      f"dp = {crown.diam_pitch * 1.0e3:.2f} mm | "
      f"dm = {crown.diam_mean * 1.0e3:.2f} mm | "
      f"CSf = {crown.csf(mat_sigma_sf, sys_life_cycle, 25.0, sys_reliability, gear_k0, j2, torque2, rpm2):.3f}"
      f"CSp = {crown.csp(gear_k0, gear_kxc, gear_i, mat_sigma_sp, mat_youngs, sys_ratio, torque2, rpm2, rpm1, sys_life_cycle, 25.0, sys_reliability):.3f}")
print(f"Custo do par: {cost_function(pinion, sys_ratio, mat_hardness):.2f}")